In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

from sklearn.linear_model import Lasso

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')  # Suppress all warnings

## The Data: Boston house prices dataset

The dataset contain 506 towns' median house price and each town's information.  There are 13 features and 1 target (in order):
- **CRIM**     per capita crime rate by town
- **ZN**       proportion of residential land zoned for lots over 25,000 sq.ft.
- **INDUS**    proportion of non-retail business acres per town
- **CHAS**     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- **NOX**      nitric oxides concentration (parts per 10 million)
- **RM**       average number of rooms per dwelling
- **AGE**      proportion of owner-occupied units built prior to 1940
- **DIS**      weighted distances to five Boston employment centres
- **RAD**      index of accessibility to radial highways
- **TAX**      full-value property-tax rate per 10,000
- **PTRATIO**  pupil-teacher ratio by town
- **B**        1000(Bk - 0.63)^2 where Bk is the proportion of black people by town
- **LSTAT**    % lower status of the population
- **MEDV**     Median value of owner-occupied homes in 1000s (target variable)

In [ ]:
df = pd.read_csv('boston_house_prices.csv')    

df.describe()                # Need scaling? 

Split the data into Train and Test set.

In [ ]:
X = df.drop(columns = 'MEDV')

y = df['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)    

display(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 1. Linear Regression with a Pipeline

With the help of ``Pipeline``, let's build a ``Linear Regression`` with ``MinMaxScaler`` and  ``PolynomialFeatures`` applied for data preprocessing.  

In [ ]:
# Step 1: Build the pipeline

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler  
from sklearn.preprocessing import PolynomialFeatures

lr_pipe = Pipeline([("scaler", MinMaxScaler()), 
                    ("poly",PolynomialFeatures(degree = 2, include_bias = False)),  # no need get 1 for intercept 
                    ("model", LinearRegression())])    

lr_pipe.named_steps 

In [ ]:
# Step 2: Train the pipeline and check performance

lr_pipe.fit(X_train, y_train)   

print("lr_pipe Train R2: {:.2f}".format(lr_pipe.score(X_train, y_train)))   
print("lr_pipe Test R2: {:.2f}".format(lr_pipe.score(X_test, y_test)))  

Note that as ``lr_pipe`` is a ``pipeline``, we need to call each step by their step names and apply the associated methods for result.

In [ ]:
# Step 3: Get features and coefs 

poly = lr_pipe.named_steps['poly']  # Get the polynomial transformer

lr = lr_pipe.named_steps['model']         # Get the model  

pd.DataFrame({'features': poly.get_feature_names_out(), 'coefs':lr.coef_})   

# Get 104 features in total (orginally 13)
# New feature names (x0,x1..x12) were auto-generated corresponding to their original column index. 
# To check original feature names: X_train.columns

**The linear regression with too many polynomrial features overfits**

As indicated by the huge gap between the train and test R2, **the linear regression model might have overfitted**, so we need to control model complexity a bit by regularizing the coefficient magnitude.  

- [Ridge Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) regularize model complexity by penalizing the sum of squares (L2-norms) of coefficients, which will reduce feature importance (i.e., coef) to be as ``close to 0``.

- [Lasso Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) control model complexity by penalizing the sum of absolute values (L1-norms) of coefficients. As it reduce feature importance (i.e. coef) to be ``exactly 0``, it often works for automatical feature selection.

For both ``Ridge`` and ``Lasso``, the strength of penalty is controlled by the parameter ``alpha`` (default = 1). A larger ``alpha`` value means larger panelty in the objective function (which we aims to minimize) and therefore stronger regularization on the coefficients, leading to a simpler model (with smaller coefs).   

# 2. Ridge Regression

## 2.1 ``Ridge`` Models with Different Regularization Strength

With the help of pipelines, build three ``Ridge`` models with different ``alpha`` values (i.e., ``0.1``, ``1``, and ``10``).

- use ``MinMaxScaler`` and ``PolynomialFeatures`` with ``degree`` = 2. (same as ``lr_pipe``)

In [ ]:
# Step 1: build the pipelines

ridge1_pipe = Pipeline([("scaler", MinMaxScaler()), 
                        ("poly",PolynomialFeatures(degree = 2, include_bias = False)),   
                        ("model", Ridge(alpha = 0.1))])

ridge2_pipe = Pipeline([("scaler", MinMaxScaler()), 
                        ("poly",PolynomialFeatures(degree = 2, include_bias = False)),   
                        ("model", Ridge())])      # Default alpha = 1

ridge3_pipe = Pipeline([("scaler", MinMaxScaler()), 
                        ("poly",PolynomialFeatures(degree = 2, include_bias = False)),   
                        ("model", Ridge(alpha = 10))])

display(ridge1_pipe.named_steps, ridge2_pipe.named_steps, ridge3_pipe.named_steps)   

In [ ]:
# Step 2: Train the pipeline and check performance

ridge1_pipe.fit(X_train, y_train)
ridge2_pipe.fit(X_train, y_train)
ridge3_pipe.fit(X_train, y_train)

print("ridge1(alpha = 0.1) Train & Test R2: {:.2f} & {:.2f}".format(ridge1_pipe.score(X_train, y_train),
                                                                    ridge1_pipe.score(X_test, y_test)))

print("ridge2(alpha = 1) Train & Test R2: {:.2f} & {:.2f}".format(ridge2_pipe.score(X_train, y_train),
                                                                  ridge2_pipe.score(X_test, y_test)))

print("ridge3(alpha = 10) Train & Test R2: {:.2f} & {:.2f}".format(ridge3_pipe.score(X_train, y_train), 
                                                                   ridge3_pipe.score(X_test, y_test)))

Now you can see, the best ridge model is ``ridge1_pipe`` (with ``alpha`` = 0.1), which return a worse train R2 (0.93) but better test R2 (0.75) compared with ``lr_pipe`` (0.95 and 0.62 respectively). 

-  ``ridge2_pipe`` (``alpha`` = 1) is a simpler model, worse train and test R2 compared with ``ridge1_pipe`` (starting to underfit).
-  ``ridge3_pipe`` (``alpha`` = 10) is too simple, worse train and test R2 (more underfitting).

**Any change in coefficients?**

Let's get coefficients for all three models, compared their ``sum of squared coefficients`` and visualize the magnitude of coefficients. 

- Larger ``alpha`` means smaller ``sum of squared coefficients``.  
- Linear regression can be seen as regularized regression with ``alpha`` = 0.

In [ ]:
# Step3: Get features and coefs  

poly1 = ridge1_pipe.named_steps['poly']  # get the polynomial transformer  

ridge1 = ridge1_pipe.named_steps['model']      # get ridge1 model  
ridge2 = ridge2_pipe.named_steps['model']
ridge3 = ridge3_pipe.named_steps['model']

# Put all coefs in a dataframe for better visualization, which feature is most important? use .max()
pd.DataFrame({'features': poly1.get_feature_names_out(),    # same features for three models
              'ridge1_coefs':ridge1.coef_,
              'ridge2_coefs':ridge2.coef_,
              'ridge3_coefs':ridge2.coef_})      

In [ ]:
# get the sum of squared coefficients: bigger alpha, smaller sum of squared coefs

print("lr sum of squared coefs: {:.2f}".format(sum((lr.coef_)**2)))  
print("ridge1 sum of squared coefs: {:.2f}".format(sum((ridge1.coef_)**2))) 
print("ridge2 sum of squared coefs: {:.2f}".format(sum((ridge2.coef_)**2))) 
print("ridge3 sum of squared coefs: {:.2f}".format(sum((ridge3.coef_)**2))) 

**Visualize the Coefficients**

Now let's visualize the coefficients (i.e. slopes) for comparison, with linear regression's coefficients included for comparison.
- When ``alpha`` = 10, most coefs close (but not equal) to 0 (blue squares).
- Reducing ``alpha`` to 1,  magnitude of coefs increases (red crosses). 
- When ``alpha`` = 0.1 (our best model so far), more coefs with large magnitudes (green triangles).
- The coefs of Linear regression are most spread out (orange circles).

In [ ]:
plt.figure(figsize=(10,6))

# Note y value = coefficients, x values = their index in the array (auto).
plt.plot(lr.coef_, 'o', color = 'orange', label="LinearRegression")            # circle
plt.plot(ridge1.coef_, '^', color = 'lightgreen',  label="Ridge alpha=0.1")    # triangle
plt.plot(ridge2.coef_, '+', color = 'red',label="Ridge alpha=1")               # cross
plt.plot(ridge3.coef_, 's', color = 'blue', label="Ridge alpha=10")            # square

# Visualize a reference line: y = 0
xlims = plt.xlim()                                     # Get the x limits of the current plot
plt.hlines(0, xlims[0], xlims[1], color = 'grey')      
plt.ylim(-25, 25)

# Labels and title
plt.xlabel("Coefficient index")               # Order of each coef in the array
plt.ylabel("Coefficient magnitude")    
plt.title('Comparing Coefficient Magnitudes for Ridge Regressions(LR as Base)')
plt.legend();

## 2.2 ``GridSearchCV`` with a Ridge Pipeline

Then what is the best ``degree`` value for ``PolynomialFeatures()`` and the best ``alpha`` value in ``Ridge()``? We might build a pipeline for that.  

- You can use any of the above pipelines as the estimator in ``GridSearchCV`` as the param values set above will be ignored in the grid search process. 
- Pay attention to how we set the param names in the dictionary keys: step names and param names are connected with ``__`` (double underscore).

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

params1 = {'scaler':[MinMaxScaler(),StandardScaler()],      # Compare 2 scalers
           'poly__degree': [2,3,4,5],                 # Compare 4 degree values  
           'model__alpha': [0.0001,0.001,0.1,1,2]}          # Compare 5 alpha values

grid1 = GridSearchCV(estimator = ridge1_pipe, param_grid = params1, cv=5)    # Use an ridge1_pipe

grid1.fit(X_train, y_train)

print("Best Params: {}".format(grid1.best_params_))    
print("Best cross-validation R2: {}".format(grid1.best_score_))   # Best mean cv score on validation set

Check the performance of the best model, which is refit on entire train set with the best prams chosen in cross validation.

In [ ]:
test_score = grid1.score(X_test, y_test)    # Apply the best model (a pipe) on test data directly! 

print("Test score of the best model: {:.2f}".format(test_score))   # Even better than best ridge1

Check the coefficients of the best model.

- Note the best model is a pipeline, not a regression model.  You need to call each step by their names and apply the associated methods. 

In [ ]:
best_pipe = grid1.best_estimator_                 # This is a pipeline

best_poly = best_pipe.named_steps['poly']   # Get best poly transformer

best_model = best_pipe.named_steps['model']       # Get best model

print("best_pipe Sum of squared coefs: ", sum((best_model.coef_)**2))  # Get the sum of  coefs

pd.DataFrame({'grid1_features': best_poly.get_feature_names_out(), 'coefs':best_model.coef_})   # 559 features 

# 3. Lasso Regression

## 3.1  ``Lasso`` Models with Different Regularization Strength

For ``Lasso`` regression, let's build 3 pipelines with different ``alpha`` values(``1``,`` 0.01``, ``0.0001``).

- Same as above, let's use ``MinMaxScaler`` and ``PolynomialFeatures`` with ``degree`` = 2. 

In [ ]:
# degree = 2, alpha = 1 (bad performance on both train or test - underfitting)

lasso1_pipe = Pipeline([("scaler", MinMaxScaler()), 
                        ("poly",PolynomialFeatures(degree = 2, include_bias = False)), 
                        ("model", Lasso())])     # default alpha = 1
build the pipelines
lasso1_pipe.fit(X_train, y_train)   

print("lasso1 (alpha = 1) Train R2: {:.2f}".format(lasso1_pipe.score(X_train, y_train)))   
print("lasso1 (alpha = 1) Test R2: {:.2f}".format(lasso1_pipe.score(X_test, y_test)))   

<font color=red>***Exercise 1: Your Codes Here***</font>  

``lasso1_pipe`` is obviously underfitting, so we need to tune down the regularization strength a bit.   Please build ``lasso2_pipe`` and ``lasso3_pipe`` with ``alpha`` = ``0.01`` and ``0.0001`` respectively. For ``scaler`` and ``poly`` steps, use the same param as in ``lasso1_pipe``.

- Check their performance on train and test as well.
- Among all three lasso models, which one is the best one? 
- If we compare the best lasso model with the best ridge (i.e., ``ridge1_pipe``), which one is better? 

In [ ]:
# degree = 2, alpha = 0.01 (tune down regularization a bit, better result)


In [ ]:
# degree = 2, alpha = 0.0001 (start to overfit as test R2 drop)


**Any change in coefficients?**

Let's get coefficients for all three models, compared their ``sum of absolute coefficients`` and visualize the magnitude of coefficients. 

- The larger ``alpha`` (stronger regulation) is, the smaller the ``sum of absolute cofficients`` is. 

In [ ]:
lasso1_coefs = lasso1_pipe.named_steps['model'].coef_
lasso2_coefs = lasso2_pipe.named_steps['model'].coef_
lasso3_coefs = lasso3_pipe.named_steps['model'].coef_

print("lasso1 (alpha = 1) sum of absolute coefficients: {:.2f} ".format(sum(abs(lasso1_coefs))))
print("lasso2 (alpha = 0.01) sum of absolute coefficients:{:.2f} ".format(sum(abs(lasso2_coefs)))) 
print("lasso3 (alpha = 0.0001) sum of absolute coefficients: {:.2f}".format(sum(abs(lasso3_coefs))))   

- How many coefficients are useful (not equal to 0) for prediction in each model? 
- The larger ``alpha`` (stronger regulation) is, the more coefficients == 0, which means less feature will be used.

In [ ]:
print("lasso1 (alpha = 1) No. of features used:", sum(lasso1_coefs != 0))
print("lasso2 (alpha = 0.01) No. of features used:", sum(lasso2_coefs != 0))
print("lasso3 (alpha = 0.0001) No. of features used:", sum(lasso3_coefs != 0))

**Visualize the Coefficients**

- Include the best ridge model ``ridge1`` for reference (the orange circles, most coef non-zero).
-  When ``alpha`` = 1, not only most coeffs are zero, the 3 non-zero coefs are very small (blue squares).
- ``alpha`` = 0.01, more coef become non-zero, still lots are zero (red crosses) .
- ``alpha`` = 0.0001, less regularization, most coef have larger magnititude (green trianges)


In [ ]:
plt.figure(figsize=(10,6))

plt.plot(ridge1.coef_, 'o', color = 'orange',  label="Ridge alpha = 0.1")          # circles
plt.plot(lasso1_coefs, 's', color = 'blue', label="Lasso1 alpha = 1")              # squares
plt.plot(lasso2_coefs, '+', color = 'red',  label="Lasso2 alpha = 0.01")           # crosses
plt.plot(lasso3_coefs, '^', color = 'lightgreen',label="Lasso3 alpha = 0.0001")    # triangle

# Visualize a reference line: y = 0
xlims = plt.xlim()                                     
plt.hlines(0, xlims[0], xlims[1],color = 'grey')      # reference line: y = 0
plt.ylim(-25, 25)

# Labels and title
plt.xlabel("Coefficient index")                       
plt.ylabel("Coefficient magnitude")  
plt.title('Comparing Coefficient Magnitudes for Lasso Regressions (Ridge as Base)')
plt.legend();


## 3.2 ``GridSearchCV`` with a Lasso Pipeline 

Compare ``MinMaxScaler`` and ``StandardScaler``;  compare four values ``2,3,4,5`` for ``degree`` in ``PolynomialFeatures`` and four values ``0.0001, 0.0005, 0.001, 0.1`` for ``alpha`` in the ``Lasso`` model. (As we know ``alpha`` = 0.0001 causes overfitting already.)

*Hints: You may use any of the pipeline built in section 3.1 as the parameter set above will be ignored in ``GridSearchCV``.*

- Return the best params for each step and the cross-validation performance on validation fold.
- Check the generalization performance of the best model (refit on the train set) on the test set.
- Check how many coefficients returned by the best model are NOT zero? 

<font color=red>***Exercise 2: Your Codes Here***</font>  

In [ ]:
# test R2 of best model, similiar to the best ridge via Gridsearch (test R2: 0.78)

grid2.score(X_test, y_test)

In [ ]:
best_pipe = grid2.best_estimator_      

sum(best_pipe.named_steps['model'].coef_ != 0)   # a lot! as alpha = 0.001 and degree = 3

#len(best_pipe.named_steps['model'].coef_)       # Total number of coef is 559  